In [1]:
from dotenv import load_dotenv
import os
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Initialize LLM models

### LLama by Ollama

In [2]:
# setup for llama3
ollama_llm = Ollama(model='llama3')   

In [ ]:
ollama_llm.invoke("What is a bot?")

### ChatGPT by OpenAI

In [6]:
# setup GPT
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'
gpt_llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=Model)

In [ ]:
gpt_llm.invoke('What is a bot?')

In [ ]:
parser = StrOutputParser()
gpt_chain = gpt_llm|parser
gpt_chain.invoke('What is a bot?')

### Gemini by google ai studio

In [1]:
from dotenv import load_dotenv
import google.generativeai as gen_ai
import os 

load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

c:\Users\Administrator\Desktop\ChatBot\venv-bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up Google Gemini-Pro AI model
gen_ai.configure(api_key=GOOGLE_API_KEY)
model = gen_ai.GenerativeModel('gemini-pro')

In [ ]:
model.start_chat()

### Llama3 by hugging face

In [1]:
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from huggingface_hub import InferenceApi
from dotenv import load_dotenv

c:\Users\Administrator\Desktop\ChatBot\venv-bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
HUGGINGFACE_HUB_TOKEN = os.getenv('HUGGINGFACE_HUB_TOKEN')
model = "meta-llama/Meta-Llama-3.1-405B"
llama_llm = HuggingFaceHub(
    repo_id= model,
    huggingfacehub_api_token=HUGGINGFACE_HUB_TOKEN
)

In [ ]:
llama_llm.invoke('what is a bot?')

In [ ]:
parser = StrOutputParser()
llama_chain = llama_llm|parser
llama_chain.invoke('What is a bot?')

# Load the vector store
Load the vector store containing client specific information to enable personalized chatbot experience

In [2]:
loader = TextLoader('../dataset/data.txt', encoding='utf-8') # saving the model using an encoding

# load all the data
document = loader.load()  

In [3]:
# split data
splitter = RecursiveCharacterTextSplitter(document=document,chunk_size=200)
chunks = splitter.split_documents(document)

In [ ]:
# convert the chunks into embeddings and embed into the vector storage
vector_storage = FAISS.from_documents(chunks, OpenAIEmbeddings())

# to retrieve the personalized answer to the prompt
retriever  = vector_storage.as_retriever()

In [ ]:
# find content in our db that relates to the question asked
retriever.invoke('What is the pricing for bhuman')

# Propmt prompt_template

In [8]:
template = ("""
            You are AI powered chatbot designed to provide 
            information and assistance for customers based on the context
            provided to you only. Do not make anything up.
            Only use the context you are provided.
            
            context: {context}
            Question: {question}
            """)

In [12]:
template.format(
    context='conteadfadfa',
    question= '?'
)

'\n            You are AI powered chatbot designed to provide \n            information and assistance for customers based on the context\n            provided to you only. Do not make anything up.\n            Only use the context you are provided.\n            \n            context: conteadfadfa\n            Question: ?\n            '

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt = PromptTemplate.from_tempate(template=template)
prompt.format(
    context = 'Here is the context',
    question = 'This is a question to answer'
)

#### The prompt goes to the vector store and the LLM. 

In [ ]:
# give it the context from the vector store 
result = RunnableParallel(context = retriever, 
                          question=RunnablePassthrough 
                          )

In [ ]:
# output parser
parser = StrOutputParser()

#### ChatGPT

In [ ]:
chain = result | prompt | gpt_llm | parser  # chain the result, prmpt, gpt_llm and parse theresponse
chain.invoke('What is the pricing plan for Bhuman')

#### LLama

In [ ]:
chain = result | prompt | llama_llm | parser  # chain the result, prmpt, gpt_llm and parse theresponse
chain.invoke('What is the pricing plan for Bhuman')